In [1]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout
from keras.optimizers import RMSprop
from random import randint
from keras.callbacks import CSVLogger, TensorBoard
import os

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 50
activations = ['sigmoid', 'tanh', 'relu', 'linear', 'elu', 'selu']
activations_sigmoid = ['sigmoid', 'tanh']
activations_linear = ['relu', 'linear', 'elu', 'selu']
units = 64
experiments = 10
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
save_dir = os.path.join(os.getcwd(), 'saved_models')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [4]:
print('-'*30)
print('Randomized all activations')

for ex in range(experiments):
    model_name = 'mnist_rand_all_' + str(ex)
    inputs = Input(shape=(784,))
    layers_1 = []
    for i in range(units):
        activation = activations[randint(0, len(activations)-1)]
        layers_1.append(Dense(1, activation=activation)(inputs))

    x = keras.layers.concatenate(layers_1)    

    x = Dropout(0.2)(x)
    layers_2 = []
    for i in range(units):
        activation = activations[randint(0, len(activations)-1)]
        layers_2.append(Dense(1, activation=activation)(x))
    x = keras.layers.concatenate(layers_2)    
    x = Dropout(0.2)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    
    csv_logger = CSVLogger('../logs/mnist/rand_act_all_log_%d_%d.csv' % (units, ex), append=True, separator=';')
    tb = TensorBoard(log_dir='./tb_logs/rand_all_' + str(ex), histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])
    
    print('-'*30)
    print('Experiment', ex)
    
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name + str(ex) + ".h5")
    model.save(model_path)
    
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test), callbacks=[csv_logger, tb])
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

------------------------------
Randomized all activations
------------------------------
Experiment 0
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 11s 186us/step - loss: 0.4674 - acc: 0.8605 - val_loss: 0.2338 - val_acc: 0.9312
Epoch 2/50
60000/60000 [==============================] - 7s 119us/step - loss: 0.2793 - acc: 0.9173 - val_loss: 0.1930 - val_acc: 0.9444
Epoch 3/50
60000/60000 [==============================] - 7s 119us/step - loss: 0.2318 - acc: 0.9306 - val_loss: 0.1624 - val_acc: 0.9500
Epoch 4/50
60000/60000 [==============================] - 7s 120us/step - loss: 0.2051 - acc: 0.9389 - val_loss: 0.1388 - val_acc: 0.9570
Epoch 5/50
60000/60000 [==============================] - 7s 121us/step - loss: 0.1853 - acc: 0.9440 - val_loss: 0.1264 - val_acc: 0.9597
Epoch 6/50
60000/60000 [==============================] - 7s 121us/step - loss: 0.1730 - acc: 0.9478 - val_loss: 0.1205 - val_acc: 0.9631
Epoch 7/50
60000/60

Epoch 8/50
60000/60000 [==============================] - 7s 117us/step - loss: 0.1519 - acc: 0.9534 - val_loss: 0.1117 - val_acc: 0.9661
Epoch 9/50
60000/60000 [==============================] - 9s 158us/step - loss: 0.1415 - acc: 0.9562 - val_loss: 0.1047 - val_acc: 0.9689
Epoch 10/50
60000/60000 [==============================] - 11s 176us/step - loss: 0.1391 - acc: 0.9575 - val_loss: 0.1017 - val_acc: 0.9686
Epoch 11/50
60000/60000 [==============================] - 7s 123us/step - loss: 0.1320 - acc: 0.9594 - val_loss: 0.1013 - val_acc: 0.9689
Epoch 12/50
60000/60000 [==============================] - 9s 145us/step - loss: 0.1250 - acc: 0.9622 - val_loss: 0.0972 - val_acc: 0.9715
Epoch 13/50
60000/60000 [==============================] - 7s 117us/step - loss: 0.1212 - acc: 0.9623 - val_loss: 0.0926 - val_acc: 0.9716
Epoch 14/50
60000/60000 [==============================] - 10s 160us/step - loss: 0.1184 - acc: 0.9643 - val_loss: 0.0936 - val_acc: 0.9732
Epoch 15/50
60000/60000 [==

Epoch 16/50
60000/60000 [==============================] - 6s 94us/step - loss: 0.1132 - acc: 0.9643 - val_loss: 0.0962 - val_acc: 0.9724
Epoch 17/50
60000/60000 [==============================] - 6s 94us/step - loss: 0.1097 - acc: 0.9658 - val_loss: 0.0911 - val_acc: 0.9732
Epoch 18/50
60000/60000 [==============================] - 6s 95us/step - loss: 0.1066 - acc: 0.9674 - val_loss: 0.0890 - val_acc: 0.9744
Epoch 19/50
60000/60000 [==============================] - 5s 90us/step - loss: 0.1017 - acc: 0.9683 - val_loss: 0.0920 - val_acc: 0.9735
Epoch 20/50
60000/60000 [==============================] - 5s 91us/step - loss: 0.1018 - acc: 0.9690 - val_loss: 0.0883 - val_acc: 0.9747
Epoch 21/50
60000/60000 [==============================] - 5s 90us/step - loss: 0.1000 - acc: 0.9688 - val_loss: 0.0870 - val_acc: 0.9752
Epoch 22/50
60000/60000 [==============================] - 5s 90us/step - loss: 0.0984 - acc: 0.9694 - val_loss: 0.0886 - val_acc: 0.9744
Epoch 23/50
60000/60000 [=========

60000/60000 [==============================] - 6s 100us/step - loss: 0.0979 - acc: 0.9697 - val_loss: 0.0861 - val_acc: 0.9752
Epoch 25/50
60000/60000 [==============================] - 6s 94us/step - loss: 0.0984 - acc: 0.9695 - val_loss: 0.0872 - val_acc: 0.9743
Epoch 26/50
60000/60000 [==============================] - 6s 92us/step - loss: 0.0956 - acc: 0.9705 - val_loss: 0.0855 - val_acc: 0.9753
Epoch 27/50
60000/60000 [==============================] - 6s 93us/step - loss: 0.0944 - acc: 0.9716 - val_loss: 0.0902 - val_acc: 0.9743
Epoch 28/50
60000/60000 [==============================] - 6s 93us/step - loss: 0.0928 - acc: 0.9709 - val_loss: 0.0853 - val_acc: 0.9751
Epoch 29/50
60000/60000 [==============================] - 6s 93us/step - loss: 0.0890 - acc: 0.9720 - val_loss: 0.0861 - val_acc: 0.9757
Epoch 30/50
60000/60000 [==============================] - 6s 93us/step - loss: 0.0896 - acc: 0.9721 - val_loss: 0.0870 - val_acc: 0.9761
Epoch 31/50
60000/60000 [====================

60000/60000 [==============================] - 5s 88us/step - loss: 0.0889 - acc: 0.9726 - val_loss: 0.0880 - val_acc: 0.9758
Epoch 33/50
60000/60000 [==============================] - 5s 88us/step - loss: 0.0862 - acc: 0.9728 - val_loss: 0.0878 - val_acc: 0.9752
Epoch 34/50
60000/60000 [==============================] - 5s 85us/step - loss: 0.0847 - acc: 0.9733 - val_loss: 0.0880 - val_acc: 0.9757
Epoch 35/50
60000/60000 [==============================] - 5s 86us/step - loss: 0.0844 - acc: 0.9742 - val_loss: 0.0902 - val_acc: 0.9754
Epoch 36/50
60000/60000 [==============================] - 5s 86us/step - loss: 0.0840 - acc: 0.9736 - val_loss: 0.0903 - val_acc: 0.9750
Epoch 37/50
60000/60000 [==============================] - 5s 85us/step - loss: 0.0807 - acc: 0.9743 - val_loss: 0.0886 - val_acc: 0.9758
Epoch 38/50
60000/60000 [==============================] - 5s 85us/step - loss: 0.0823 - acc: 0.9751 - val_loss: 0.0908 - val_acc: 0.9768
Epoch 39/50
60000/60000 [=====================

60000/60000 [==============================] - 6s 96us/step - loss: 0.0754 - acc: 0.9770 - val_loss: 0.0917 - val_acc: 0.9755
Epoch 41/50
60000/60000 [==============================] - 6s 96us/step - loss: 0.0742 - acc: 0.9768 - val_loss: 0.0918 - val_acc: 0.9753
Epoch 42/50
60000/60000 [==============================] - 6s 96us/step - loss: 0.0739 - acc: 0.9765 - val_loss: 0.0897 - val_acc: 0.9770
Epoch 43/50
60000/60000 [==============================] - 6s 96us/step - loss: 0.0754 - acc: 0.9755 - val_loss: 0.0923 - val_acc: 0.9774
Epoch 44/50
60000/60000 [==============================] - 6s 97us/step - loss: 0.0706 - acc: 0.9775 - val_loss: 0.0944 - val_acc: 0.9756
Epoch 45/50
60000/60000 [==============================] - 6s 95us/step - loss: 0.0708 - acc: 0.9775 - val_loss: 0.0913 - val_acc: 0.9766
Epoch 46/50
60000/60000 [==============================] - 6s 96us/step - loss: 0.0697 - acc: 0.9782 - val_loss: 0.0973 - val_acc: 0.9756
Epoch 47/50
60000/60000 [=====================

60000/60000 [==============================] - 6s 94us/step - loss: 0.0721 - acc: 0.9773 - val_loss: 0.0930 - val_acc: 0.9766
Epoch 49/50
60000/60000 [==============================] - 6s 94us/step - loss: 0.0737 - acc: 0.9761 - val_loss: 0.0876 - val_acc: 0.9764
Epoch 50/50
60000/60000 [==============================] - 6s 104us/step - loss: 0.0709 - acc: 0.9774 - val_loss: 0.0862 - val_acc: 0.9768
Test loss: 0.0861615305257
Test accuracy: 0.9768
------------------------------
Experiment 7
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 7s 120us/step - loss: 0.4652 - acc: 0.8603 - val_loss: 0.2381 - val_acc: 0.9293
Epoch 2/50
60000/60000 [==============================] - 6s 99us/step - loss: 0.2758 - acc: 0.9159 - val_loss: 0.1841 - val_acc: 0.9445
Epoch 3/50
60000/60000 [==============================] - 6s 99us/step - loss: 0.2301 - acc: 0.9317 - val_loss: 0.1545 - val_acc: 0.9533
Epoch 4/50
60000/60000 [==================

60000/60000 [==============================] - 6s 104us/step - loss: 0.1816 - acc: 0.9443 - val_loss: 0.1229 - val_acc: 0.9606
Epoch 6/50
60000/60000 [==============================] - 6s 104us/step - loss: 0.1677 - acc: 0.9493 - val_loss: 0.1149 - val_acc: 0.9645
Epoch 7/50
60000/60000 [==============================] - 6s 104us/step - loss: 0.1548 - acc: 0.9523 - val_loss: 0.1056 - val_acc: 0.9671
Epoch 8/50
60000/60000 [==============================] - 6s 106us/step - loss: 0.1461 - acc: 0.9549 - val_loss: 0.1100 - val_acc: 0.9669
Epoch 9/50
60000/60000 [==============================] - 6s 105us/step - loss: 0.1381 - acc: 0.9570 - val_loss: 0.1022 - val_acc: 0.9695
Epoch 10/50
60000/60000 [==============================] - 6s 106us/step - loss: 0.1338 - acc: 0.9593 - val_loss: 0.0992 - val_acc: 0.9702
Epoch 11/50
60000/60000 [==============================] - 6s 106us/step - loss: 0.1258 - acc: 0.9617 - val_loss: 0.0961 - val_acc: 0.9704
Epoch 12/50
60000/60000 [==================

60000/60000 [==============================] - 7s 118us/step - loss: 0.1201 - acc: 0.9639 - val_loss: 0.0983 - val_acc: 0.9710
Epoch 14/50
60000/60000 [==============================] - 7s 118us/step - loss: 0.1180 - acc: 0.9642 - val_loss: 0.0972 - val_acc: 0.9718
Epoch 15/50
60000/60000 [==============================] - 7s 118us/step - loss: 0.1138 - acc: 0.9652 - val_loss: 0.1017 - val_acc: 0.9710
Epoch 16/50
60000/60000 [==============================] - 7s 117us/step - loss: 0.1116 - acc: 0.9660 - val_loss: 0.0942 - val_acc: 0.9725
Epoch 17/50
60000/60000 [==============================] - 6s 108us/step - loss: 0.1079 - acc: 0.9669 - val_loss: 0.0972 - val_acc: 0.9705
Epoch 18/50
60000/60000 [==============================] - 7s 109us/step - loss: 0.1062 - acc: 0.9673 - val_loss: 0.0926 - val_acc: 0.9732
Epoch 19/50
60000/60000 [==============================] - 6s 107us/step - loss: 0.1053 - acc: 0.9681 - val_loss: 0.0898 - val_acc: 0.9743
Epoch 20/50
60000/60000 [==============

In [5]:
print('-'*30)
print('Randomized linear activations')

for ex in range(experiments):
    model_name = 'mnist_rand_lin_' + str(ex)
    inputs = Input(shape=(784,))
    layers_1 = []
    for i in range(units):
        activation = activations_linear[randint(0, len(activations_linear)-1)]
        layers_1.append(Dense(1, activation=activation)(inputs))

    x = keras.layers.concatenate(layers_1)    

    x = Dropout(0.2)(x)
    layers_2 = []
    for i in range(units):
        activation = activations_linear[randint(0, len(activations_linear)-1)]
        layers_2.append(Dense(1, activation=activation)(x))
    x = keras.layers.concatenate(layers_2)    
    x = Dropout(0.2)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)

    csv_logger = CSVLogger('../logs/mnist/rand_act_lin_log_%d_%d.csv' % (units, ex), append=True, separator=';')
    tb = TensorBoard(log_dir='./tb_logs/rand_lin_' + str(ex), histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])
    
    print('-'*30)
    print('Experiment', ex)
    
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name + str(ex) + ".h5")
    model.save(model_path)
    
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test), callbacks=[csv_logger, tb])
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

------------------------------
Randomized linear activations
------------------------------
Experiment 0
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 9s 146us/step - loss: 0.4525 - acc: 0.8636 - val_loss: 0.2335 - val_acc: 0.9314
Epoch 2/50
60000/60000 [==============================] - 7s 117us/step - loss: 0.2703 - acc: 0.9204 - val_loss: 0.1719 - val_acc: 0.9475
Epoch 3/50
60000/60000 [==============================] - 7s 116us/step - loss: 0.2249 - acc: 0.9321 - val_loss: 0.1502 - val_acc: 0.9562
Epoch 4/50
60000/60000 [==============================] - 7s 115us/step - loss: 0.1971 - acc: 0.9407 - val_loss: 0.1275 - val_acc: 0.9606
Epoch 5/50
60000/60000 [==============================] - 7s 117us/step - loss: 0.1765 - acc: 0.9466 - val_loss: 0.1193 - val_acc: 0.9628
Epoch 6/50
60000/60000 [==============================] - 7s 117us/step - loss: 0.1657 - acc: 0.9490 - val_loss: 0.1132 - val_acc: 0.9663
Epoch 7/50
60000/

Epoch 8/50
60000/60000 [==============================] - 7s 117us/step - loss: 0.1481 - acc: 0.9546 - val_loss: 0.1072 - val_acc: 0.9686
Epoch 9/50
60000/60000 [==============================] - 7s 119us/step - loss: 0.1438 - acc: 0.9557 - val_loss: 0.0982 - val_acc: 0.9699
Epoch 10/50
60000/60000 [==============================] - 7s 119us/step - loss: 0.1357 - acc: 0.9587 - val_loss: 0.1022 - val_acc: 0.9681
Epoch 11/50
60000/60000 [==============================] - 7s 118us/step - loss: 0.1313 - acc: 0.9605 - val_loss: 0.0982 - val_acc: 0.9711
Epoch 12/50
60000/60000 [==============================] - 7s 118us/step - loss: 0.1277 - acc: 0.9614 - val_loss: 0.0955 - val_acc: 0.9720
Epoch 13/50
60000/60000 [==============================] - 7s 118us/step - loss: 0.1239 - acc: 0.9624 - val_loss: 0.0962 - val_acc: 0.9722
Epoch 14/50
60000/60000 [==============================] - 7s 118us/step - loss: 0.1214 - acc: 0.9621 - val_loss: 0.1032 - val_acc: 0.9698
Epoch 15/50
60000/60000 [====

60000/60000 [==============================] - 7s 122us/step - loss: 0.1180 - acc: 0.9639 - val_loss: 0.0919 - val_acc: 0.9735
Epoch 17/50
60000/60000 [==============================] - 7s 122us/step - loss: 0.1152 - acc: 0.9647 - val_loss: 0.0891 - val_acc: 0.9738
Epoch 18/50
60000/60000 [==============================] - 7s 124us/step - loss: 0.1108 - acc: 0.9661 - val_loss: 0.0924 - val_acc: 0.9726
Epoch 19/50
60000/60000 [==============================] - 7s 122us/step - loss: 0.1080 - acc: 0.9674 - val_loss: 0.0904 - val_acc: 0.9725
Epoch 20/50
60000/60000 [==============================] - 7s 122us/step - loss: 0.1084 - acc: 0.9669 - val_loss: 0.0914 - val_acc: 0.9739
Epoch 21/50
60000/60000 [==============================] - 8s 126us/step - loss: 0.1068 - acc: 0.9675 - val_loss: 0.0893 - val_acc: 0.9746
Epoch 22/50
60000/60000 [==============================] - 8s 139us/step - loss: 0.1050 - acc: 0.9671 - val_loss: 0.0915 - val_acc: 0.9738
Epoch 23/50
60000/60000 [==============

60000/60000 [==============================] - 8s 130us/step - loss: 0.0964 - acc: 0.9700 - val_loss: 0.0985 - val_acc: 0.9736
Epoch 25/50
60000/60000 [==============================] - 8s 129us/step - loss: 0.0986 - acc: 0.9699 - val_loss: 0.0920 - val_acc: 0.9758
Epoch 26/50
60000/60000 [==============================] - 8s 129us/step - loss: 0.0968 - acc: 0.9701 - val_loss: 0.0905 - val_acc: 0.9766
Epoch 27/50
60000/60000 [==============================] - 8s 133us/step - loss: 0.0952 - acc: 0.9710 - val_loss: 0.0949 - val_acc: 0.9756
Epoch 28/50
60000/60000 [==============================] - 8s 130us/step - loss: 0.0976 - acc: 0.9701 - val_loss: 0.0929 - val_acc: 0.9757
Epoch 29/50
60000/60000 [==============================] - 8s 129us/step - loss: 0.0916 - acc: 0.9719 - val_loss: 0.0928 - val_acc: 0.9755
Epoch 30/50
60000/60000 [==============================] - 8s 129us/step - loss: 0.0935 - acc: 0.9716 - val_loss: 0.0954 - val_acc: 0.9746
Epoch 31/50
60000/60000 [==============

60000/60000 [==============================] - 8s 141us/step - loss: 0.0881 - acc: 0.9726 - val_loss: 0.0861 - val_acc: 0.9756
Epoch 33/50
60000/60000 [==============================] - 8s 141us/step - loss: 0.0885 - acc: 0.9729 - val_loss: 0.0852 - val_acc: 0.9760
Epoch 34/50
60000/60000 [==============================] - 8s 141us/step - loss: 0.0851 - acc: 0.9735 - val_loss: 0.0898 - val_acc: 0.9770
Epoch 35/50
60000/60000 [==============================] - 9s 142us/step - loss: 0.0855 - acc: 0.9732 - val_loss: 0.0854 - val_acc: 0.9761
Epoch 36/50
60000/60000 [==============================] - 9s 142us/step - loss: 0.0835 - acc: 0.9739 - val_loss: 0.0918 - val_acc: 0.9752
Epoch 37/50
60000/60000 [==============================] - 8s 140us/step - loss: 0.0828 - acc: 0.9742 - val_loss: 0.0906 - val_acc: 0.9765
Epoch 38/50
60000/60000 [==============================] - 8s 141us/step - loss: 0.0802 - acc: 0.9749 - val_loss: 0.0942 - val_acc: 0.9763
Epoch 39/50
60000/60000 [==============

60000/60000 [==============================] - 9s 158us/step - loss: 0.0865 - acc: 0.9734 - val_loss: 0.0856 - val_acc: 0.9769
Epoch 41/50
60000/60000 [==============================] - 9s 149us/step - loss: 0.0863 - acc: 0.9733 - val_loss: 0.0824 - val_acc: 0.9780
Epoch 42/50
60000/60000 [==============================] - 9s 147us/step - loss: 0.0836 - acc: 0.9739 - val_loss: 0.0840 - val_acc: 0.9778
Epoch 43/50
60000/60000 [==============================] - 9s 146us/step - loss: 0.0858 - acc: 0.9735 - val_loss: 0.0855 - val_acc: 0.9783
Epoch 44/50
60000/60000 [==============================] - 9s 147us/step - loss: 0.0827 - acc: 0.9741 - val_loss: 0.0840 - val_acc: 0.9779
Epoch 45/50
60000/60000 [==============================] - 9s 147us/step - loss: 0.0831 - acc: 0.9746 - val_loss: 0.0902 - val_acc: 0.9762
Epoch 46/50
60000/60000 [==============================] - 9s 147us/step - loss: 0.0819 - acc: 0.9746 - val_loss: 0.0867 - val_acc: 0.9783
Epoch 47/50
60000/60000 [==============

Epoch 48/50
60000/60000 [==============================] - 10s 160us/step - loss: 0.0795 - acc: 0.9762 - val_loss: 0.0984 - val_acc: 0.9764
Epoch 49/50
60000/60000 [==============================] - 10s 166us/step - loss: 0.0811 - acc: 0.9749 - val_loss: 0.0972 - val_acc: 0.9765
Epoch 50/50
60000/60000 [==============================] - 10s 166us/step - loss: 0.0798 - acc: 0.9752 - val_loss: 0.0971 - val_acc: 0.9758
Test loss: 0.0971105349942
Test accuracy: 0.9758
------------------------------
Experiment 7
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 15s 244us/step - loss: 0.4452 - acc: 0.8655 - val_loss: 0.2336 - val_acc: 0.9319
Epoch 2/50
60000/60000 [==============================] - 10s 164us/step - loss: 0.2730 - acc: 0.9177 - val_loss: 0.1774 - val_acc: 0.9462
Epoch 3/50
60000/60000 [==============================] - 10s 164us/step - loss: 0.2263 - acc: 0.9316 - val_loss: 0.1551 - val_acc: 0.9531
Epoch 4/50
60000/600

60000/60000 [==============================] - 10s 173us/step - loss: 0.1986 - acc: 0.9406 - val_loss: 0.1445 - val_acc: 0.9551
Epoch 5/50
60000/60000 [==============================] - 11s 176us/step - loss: 0.1826 - acc: 0.9449 - val_loss: 0.1283 - val_acc: 0.9617
Epoch 6/50
60000/60000 [==============================] - 10s 173us/step - loss: 0.1696 - acc: 0.9485 - val_loss: 0.1157 - val_acc: 0.9653
Epoch 7/50
60000/60000 [==============================] - 10s 171us/step - loss: 0.1601 - acc: 0.9508 - val_loss: 0.1124 - val_acc: 0.9659
Epoch 8/50
60000/60000 [==============================] - 10s 171us/step - loss: 0.1517 - acc: 0.9535 - val_loss: 0.1117 - val_acc: 0.9647
Epoch 9/50
60000/60000 [==============================] - 10s 174us/step - loss: 0.1437 - acc: 0.9565 - val_loss: 0.1007 - val_acc: 0.9694
Epoch 10/50
60000/60000 [==============================] - 10s 173us/step - loss: 0.1349 - acc: 0.9585 - val_loss: 0.1001 - val_acc: 0.9700
Epoch 11/50
60000/60000 [============

Epoch 12/50
60000/60000 [==============================] - 12s 202us/step - loss: 0.1222 - acc: 0.9625 - val_loss: 0.0977 - val_acc: 0.9722
Epoch 13/50
60000/60000 [==============================] - 12s 207us/step - loss: 0.1209 - acc: 0.9638 - val_loss: 0.0926 - val_acc: 0.9751
Epoch 14/50
60000/60000 [==============================] - 12s 203us/step - loss: 0.1194 - acc: 0.9632 - val_loss: 0.0979 - val_acc: 0.9726
Epoch 15/50
60000/60000 [==============================] - 11s 181us/step - loss: 0.1122 - acc: 0.9654 - val_loss: 0.0933 - val_acc: 0.9741
Epoch 16/50
60000/60000 [==============================] - 11s 182us/step - loss: 0.1117 - acc: 0.9655 - val_loss: 0.0955 - val_acc: 0.9730
Epoch 17/50
60000/60000 [==============================] - 11s 181us/step - loss: 0.1112 - acc: 0.9668 - val_loss: 0.0954 - val_acc: 0.9740
Epoch 18/50
60000/60000 [==============================] - 11s 181us/step - loss: 0.1074 - acc: 0.9675 - val_loss: 0.0923 - val_acc: 0.9760
Epoch 19/50
60000/60

In [6]:
print('-'*30)
print('Randomized sigmoid activations')

for ex in range(experiments):
    model_name = 'mnist_rand_sig_' + str(ex)
    inputs = Input(shape=(784,))
    layers_1 = []
    for i in range(units):
        activation = activations_sigmoid[randint(0, len(activations_sigmoid)-1)]
        layers_1.append(Dense(1, activation=activation)(inputs))

    x = keras.layers.concatenate(layers_1)    

    x = Dropout(0.2)(x)
    layers_2 = []
    for i in range(units):
        activation = activations_sigmoid[randint(0, len(activations_sigmoid)-1)]
        layers_2.append(Dense(1, activation=activation)(x))
    x = keras.layers.concatenate(layers_2)    
    x = Dropout(0.2)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)

    csv_logger = CSVLogger('../logs/mnist/rand_act_sig_log_%d_%d.csv' % (units, ex), append=True, separator=';')
    tb = TensorBoard(log_dir='./tb_logs/rand_sig_' + str(ex), histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])
    
    print('-'*30)
    print('Experiment', ex)
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name + str(ex) + ".h5")
    model.save(model_path)
        
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test), callbacks=[csv_logger, tb])
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

------------------------------
Randomized sigmoid activations
------------------------------
Experiment 0
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 19s 323us/step - loss: 0.5698 - acc: 0.8411 - val_loss: 0.2713 - val_acc: 0.9195
Epoch 2/50
60000/60000 [==============================] - 11s 185us/step - loss: 0.3046 - acc: 0.9092 - val_loss: 0.2198 - val_acc: 0.9330
Epoch 3/50
60000/60000 [==============================] - 11s 185us/step - loss: 0.2577 - acc: 0.9229 - val_loss: 0.1844 - val_acc: 0.9427
Epoch 4/50
60000/60000 [==============================] - 11s 186us/step - loss: 0.2233 - acc: 0.9335 - val_loss: 0.1653 - val_acc: 0.9484
Epoch 5/50
60000/60000 [==============================] - 11s 186us/step - loss: 0.2036 - acc: 0.9390 - val_loss: 0.1502 - val_acc: 0.9546
Epoch 6/50
60000/60000 [==============================] - 11s 191us/step - loss: 0.1862 - acc: 0.9448 - val_loss: 0.1450 - val_acc: 0.9576
Epoch 7/50

60000/60000 [==============================] - 13s 214us/step - loss: 0.1733 - acc: 0.9472 - val_loss: 0.1335 - val_acc: 0.9584
Epoch 8/50
60000/60000 [==============================] - 12s 208us/step - loss: 0.1638 - acc: 0.9507 - val_loss: 0.1242 - val_acc: 0.9621
Epoch 9/50
60000/60000 [==============================] - 12s 197us/step - loss: 0.1551 - acc: 0.9529 - val_loss: 0.1213 - val_acc: 0.9622
Epoch 10/50
60000/60000 [==============================] - 12s 205us/step - loss: 0.1484 - acc: 0.9555 - val_loss: 0.1148 - val_acc: 0.9661
Epoch 11/50
60000/60000 [==============================] - 12s 198us/step - loss: 0.1410 - acc: 0.9571 - val_loss: 0.1080 - val_acc: 0.9663
Epoch 12/50
60000/60000 [==============================] - 12s 201us/step - loss: 0.1348 - acc: 0.9591 - val_loss: 0.1088 - val_acc: 0.9673
Epoch 13/50
60000/60000 [==============================] - 12s 203us/step - loss: 0.1326 - acc: 0.9594 - val_loss: 0.1050 - val_acc: 0.9671
Epoch 14/50
60000/60000 [=========

60000/60000 [==============================] - 13s 215us/step - loss: 0.1245 - acc: 0.9624 - val_loss: 0.1025 - val_acc: 0.9695
Epoch 15/50
60000/60000 [==============================] - 13s 211us/step - loss: 0.1202 - acc: 0.9632 - val_loss: 0.1032 - val_acc: 0.9688
Epoch 16/50
60000/60000 [==============================] - 13s 213us/step - loss: 0.1168 - acc: 0.9649 - val_loss: 0.0980 - val_acc: 0.9691
Epoch 17/50
60000/60000 [==============================] - 13s 222us/step - loss: 0.1152 - acc: 0.9646 - val_loss: 0.0971 - val_acc: 0.9715
Epoch 18/50
60000/60000 [==============================] - 14s 228us/step - loss: 0.1136 - acc: 0.9656 - val_loss: 0.0941 - val_acc: 0.9724
Epoch 19/50
60000/60000 [==============================] - 13s 214us/step - loss: 0.1094 - acc: 0.9663 - val_loss: 0.0934 - val_acc: 0.9716
Epoch 20/50
60000/60000 [==============================] - 13s 214us/step - loss: 0.1057 - acc: 0.9679 - val_loss: 0.0935 - val_acc: 0.9728
Epoch 21/50
60000/60000 [=======

60000/60000 [==============================] - 14s 237us/step - loss: 0.1026 - acc: 0.9691 - val_loss: 0.0937 - val_acc: 0.9733
Epoch 22/50
60000/60000 [==============================] - 14s 230us/step - loss: 0.0993 - acc: 0.9693 - val_loss: 0.0918 - val_acc: 0.9739
Epoch 23/50
60000/60000 [==============================] - 13s 221us/step - loss: 0.0967 - acc: 0.9702 - val_loss: 0.0927 - val_acc: 0.9740
Epoch 24/50
60000/60000 [==============================] - 13s 221us/step - loss: 0.0953 - acc: 0.9703 - val_loss: 0.0924 - val_acc: 0.9732
Epoch 25/50
60000/60000 [==============================] - 13s 219us/step - loss: 0.0938 - acc: 0.9709 - val_loss: 0.0909 - val_acc: 0.9735
Epoch 26/50
60000/60000 [==============================] - 13s 221us/step - loss: 0.0910 - acc: 0.9722 - val_loss: 0.0923 - val_acc: 0.9740
Epoch 27/50
60000/60000 [==============================] - 13s 221us/step - loss: 0.0933 - acc: 0.9708 - val_loss: 0.0919 - val_acc: 0.9754
Epoch 28/50
60000/60000 [=======

60000/60000 [==============================] - 14s 238us/step - loss: 0.0889 - acc: 0.9727 - val_loss: 0.0929 - val_acc: 0.9728
Epoch 29/50
60000/60000 [==============================] - 15s 253us/step - loss: 0.0861 - acc: 0.9737 - val_loss: 0.0927 - val_acc: 0.9739
Epoch 30/50
60000/60000 [==============================] - 16s 266us/step - loss: 0.0858 - acc: 0.9729 - val_loss: 0.0935 - val_acc: 0.9739
Epoch 31/50
60000/60000 [==============================] - 14s 241us/step - loss: 0.0873 - acc: 0.9732 - val_loss: 0.0938 - val_acc: 0.9737
Epoch 32/50
60000/60000 [==============================] - 14s 236us/step - loss: 0.0846 - acc: 0.9736 - val_loss: 0.0898 - val_acc: 0.9744
Epoch 33/50
60000/60000 [==============================] - 14s 235us/step - loss: 0.0824 - acc: 0.9740 - val_loss: 0.0916 - val_acc: 0.9747
Epoch 34/50
60000/60000 [==============================] - 14s 238us/step - loss: 0.0808 - acc: 0.9747 - val_loss: 0.0922 - val_acc: 0.9739
Epoch 35/50
60000/60000 [=======

60000/60000 [==============================] - 15s 249us/step - loss: 0.0789 - acc: 0.9754 - val_loss: 0.0874 - val_acc: 0.9742
Epoch 36/50
60000/60000 [==============================] - 15s 252us/step - loss: 0.0783 - acc: 0.9760 - val_loss: 0.0866 - val_acc: 0.9749
Epoch 37/50
60000/60000 [==============================] - 15s 250us/step - loss: 0.0768 - acc: 0.9758 - val_loss: 0.0828 - val_acc: 0.9769
Epoch 38/50
60000/60000 [==============================] - 15s 252us/step - loss: 0.0752 - acc: 0.9766 - val_loss: 0.0836 - val_acc: 0.9750
Epoch 39/50
60000/60000 [==============================] - 15s 251us/step - loss: 0.0740 - acc: 0.9771 - val_loss: 0.0855 - val_acc: 0.9755
Epoch 40/50
60000/60000 [==============================] - 16s 267us/step - loss: 0.0733 - acc: 0.9772 - val_loss: 0.0848 - val_acc: 0.9759
Epoch 41/50
60000/60000 [==============================] - 17s 280us/step - loss: 0.0736 - acc: 0.9774 - val_loss: 0.0858 - val_acc: 0.9756
Epoch 42/50
60000/60000 [=======

60000/60000 [==============================] - 17s 282us/step - loss: 0.0746 - acc: 0.9768 - val_loss: 0.0881 - val_acc: 0.9750
Epoch 43/50
60000/60000 [==============================] - 18s 304us/step - loss: 0.0719 - acc: 0.9769 - val_loss: 0.0834 - val_acc: 0.9763
Epoch 44/50
60000/60000 [==============================] - 17s 291us/step - loss: 0.0725 - acc: 0.9774 - val_loss: 0.0852 - val_acc: 0.9773
Epoch 45/50
60000/60000 [==============================] - 17s 281us/step - loss: 0.0707 - acc: 0.9781 - val_loss: 0.0851 - val_acc: 0.9766
Epoch 46/50
60000/60000 [==============================] - 17s 282us/step - loss: 0.0709 - acc: 0.9779 - val_loss: 0.0864 - val_acc: 0.9771
Epoch 47/50
60000/60000 [==============================] - 17s 279us/step - loss: 0.0702 - acc: 0.9775 - val_loss: 0.0830 - val_acc: 0.9766
Epoch 48/50
60000/60000 [==============================] - 17s 283us/step - loss: 0.0707 - acc: 0.9780 - val_loss: 0.0847 - val_acc: 0.9772
Epoch 49/50
60000/60000 [=======

60000/60000 [==============================] - 18s 299us/step - loss: 0.0674 - acc: 0.9786 - val_loss: 0.0933 - val_acc: 0.9762
Epoch 50/50
60000/60000 [==============================] - 18s 295us/step - loss: 0.0696 - acc: 0.9780 - val_loss: 0.0929 - val_acc: 0.9764
Test loss: 0.0912338060267
Test accuracy: 0.9771
------------------------------
Experiment 8
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 31s 512us/step - loss: 0.5401 - acc: 0.8499 - val_loss: 0.2530 - val_acc: 0.9248
Epoch 2/50
60000/60000 [==============================] - 18s 304us/step - loss: 0.2845 - acc: 0.9155 - val_loss: 0.2110 - val_acc: 0.9352
Epoch 3/50
60000/60000 [==============================] - 18s 303us/step - loss: 0.2442 - acc: 0.9277 - val_loss: 0.1874 - val_acc: 0.9434
Epoch 4/50
60000/60000 [==============================] - 18s 303us/step - loss: 0.2174 - acc: 0.9349 - val_loss: 0.1623 - val_acc: 0.9516
Epoch 5/50
60000/60000 [=========

60000/60000 [==============================] - 20s 336us/step - loss: 0.2036 - acc: 0.9393 - val_loss: 0.1492 - val_acc: 0.9547
Epoch 6/50
60000/60000 [==============================] - 20s 326us/step - loss: 0.1871 - acc: 0.9442 - val_loss: 0.1383 - val_acc: 0.9594
Epoch 7/50
60000/60000 [==============================] - 19s 317us/step - loss: 0.1734 - acc: 0.9481 - val_loss: 0.1304 - val_acc: 0.9606
Epoch 8/50
60000/60000 [==============================] - 19s 317us/step - loss: 0.1641 - acc: 0.9498 - val_loss: 0.1259 - val_acc: 0.9624
Epoch 9/50
60000/60000 [==============================] - 19s 315us/step - loss: 0.1544 - acc: 0.9535 - val_loss: 0.1221 - val_acc: 0.9638
Epoch 10/50
60000/60000 [==============================] - 19s 319us/step - loss: 0.1452 - acc: 0.9563 - val_loss: 0.1163 - val_acc: 0.9654
Epoch 11/50
60000/60000 [==============================] - 19s 318us/step - loss: 0.1387 - acc: 0.9571 - val_loss: 0.1100 - val_acc: 0.9677
Epoch 12/50
60000/60000 [===========